In [ ]:
import sqlite3
from IPython.display import HTML, display

db = 'file:results.db?mode=ro'
con = sqlite3.connect(db, uri=True)

cur = con.cursor()
cur.execute('SELECT * FROM runs')

def display_result(cur, data=cur.fetchall()):
    html = "<table>"
    headers = list(map(lambda x: x[0], cur.description))
    
    html += "<thead><tr>" + "".join(["<td>" + h + "</td>" for h in headers]) + "</tr></thead>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td>%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))
display_result(cur)

In [ ]:
run_id = 25

In [ ]:
%matplotlib notebook
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import EngFormatter
from scipy.stats import linregress

# warning: not safe from SQL injection
def fetch_data(mode):
    cur.execute(f"SELECT n_summands, datafile, time_ns, stddev, repetitions FROM results WHERE mode = ?" \
                "AND run_id = ? ORDER BY n_summands",
               (mode, run_id))

def datasets():
    with sqlite3.connect('file:results.db?mode=ro', uri=True) as con:
        cur = con.cursor()

def xye(data):
    x = list(map(lambda x: x[0], data)) # n_summands
    y = list(map(lambda x: x[2] / 1e9, data)) # time_ns
    e = list(map(lambda x: x[3] / 1e9, data)) # stddev
    return x, y, e

settings = {
    "allreduce": {
        "fmt": "bo",
        "label": "MPI_Allreduce",
        "include": True,
    },
    "tree": {
        "fmt": "ro",
        "label": "Tree Reduce",
        "include": True
    },
    "reproblas": {
        "fmt": "co",
        "label": "ReproBLAS",
        "include": True
    },
    "baseline": {
        "fmt": "go",
        "label": "Sequential Baseline",
        "include": False
    }
}
linreg = False

f = plt.figure()
ax = f.subplots(1)
ax.set_ylabel('Accumulate Time')
ax.set_xlabel('number of summands')

formatter0 = EngFormatter(unit='s', places=2)
formatter1 = EngFormatter(places=2)
ax.yaxis.set_major_formatter(formatter0)
ax.xaxis.set_major_formatter(formatter1)

for mode, v in settings.items():
    if not v["include"]:
        continue
        
    fetch_data(mode)
    data = cur.fetchall()
    display(HTML(f"<h4>{mode}</h4>"))
    display_result(cur, data)
    
    ar_x, ar_y, ar_error = xye(data)
    ax.errorbar(ar_x, ar_y, yerr=ar_error, fmt=v["fmt"], label=v["label"], capsize=4.0)
    
    if linreg:
        settings[mode]["linreg"] = linregress(ar_x, ar_y)
        gradient, intercept, r_value, p_value, std_err = settings[mode]["linreg"]
        x1 = np.linspace(min(ar_x), max(ar_x), 50)
        y1 = gradient * x1 + intercept
        plt.plot(x1, y1, v["fmt"][0])

ax.legend(loc='upper left')

ax.set_xlim(left=0)
#ax.set_ylim(bottom=0)

plt.show()
plt.savefig(fname="figure.svg")



In [ ]:
def display_table2(data, header = []):
    html = "<table>"
    if len(header) > 0:
        html += "<tr>" + "".join(["<th>" + h + "</th>" for h in header])
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td>%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

if linreg:
    header = ["Mode", "Gradient", "R"]
    t = [((mode,
          settings[mode]["linreg"][0],
          settings[mode]["linreg"][2]) if settings[mode]["include"] else []) for mode in settings.keys()]
    display_table2(t, header=header)
    

In [ ]:
if linreg:
    slowdown = settings["tree"]["linreg"][0] / settings["allreduce"]["linreg"][0]
    slodown2 = settings["tree"]["linreg"][0] / settings["reproblas"]["linreg"][0]
    display(HTML(f"<h3>Tree Reduce slowdown compared to Allreduce: {round(slowdown,2)}"))
    display(HTML(f"<h3>Tree Reduce slowdown compared to ReproBLAS: {round(slodown2,2)}"))

## Violin plot

In [ ]:
def fetch_durations(run_id):
    plot_data = [[]]
    runs = cur.execute(f"SELECT id, mode, n_summands, datafile, ranks FROM results" \
                    " WHERE run_id = ? ORDER BY n_summands, datafile, mode", (run_id,))
    prev_datafile = None
    
    for result_id, mode, n_summands, datafile, ranks in runs.fetchall():
        if prev_datafile is not None and prev_datafile != datafile:
            plot_data.append([])
        cur.execute("SELECT time_ns / 1e9 FROM durations WHERE result_id = ?", (result_id,))
        
        durations = list(map(lambda x: x[0], cur.fetchall()))
        plot_data[-1].append({
            "mode": mode,
            "datafile": datafile,
            "n": n_summands,
            "m": ranks,
            "durations": durations
        })
        
        
        prev_datafile = datafile
    return plot_data

plot_data = fetch_durations(run_id)
rows = len(plot_data)
cols = max(map(lambda x: len(x), plot_data))
formatter0 = EngFormatter(unit='s', places=2)

f, axs = plt.subplots(nrows=rows, figsize=(6, 4 * rows))

i = j = 0

colorpalette = ["#034670ff", "#2f779dff", "#887006ff", "#ac360bff"]

for dataset in plot_data:
    ax = axs[i]
    
    mode_durations = list(map(lambda x: x["durations"], dataset))

    ax.set_ylabel("Time")
    ax.yaxis.set_major_formatter(formatter0)
    
    ax.get_xaxis().set_visible(True)
    
    # Ticks for each mode
    ax.get_xaxis().set_ticks(range(1, len(dataset)+1))
    ax.get_xaxis().set_ticklabels(list(map(lambda x: x["mode"], dataset)))
    
    n = dataset[0]["n"]
    m = dataset[0]["m"]
    filename = dataset[0]["datafile"]
    ax.set_title(f"{filename}, n={n}, m={m}")
    
    violins = ax.violinplot(mode_durations)
    for body, color in zip(violins["bodies"], colorpalette):
        body.set_color(color)
    
    
    i += 1